# analogy: default program

In [59]:
import sys
sys.path.append("../")
from default import *
import os
import numpy as np

## Run the default solution on dev

In [60]:
!pip install gensim

In [61]:
import gensim.downloader as api
from gensim.models import KeyedVectors

Download the glove 100d model trained on the gigaword corpus.

In [62]:
model_gigaword = api.load("glove-wiki-gigaword-100")

Get the output on the word analogy task by reading the dev set and using vector arithmetic over the word vectors for the first 10 lines in the dev set.

In [63]:
output = []
not_working = []
working = []
with open(os.path.join('../data', 'input', 'dev.txt')) as fh:
    for i, line in enumerate(fh):
        if i>10:
            break
        line = line.strip()
        if line[0] == ':':
            output.append(line)
            continue
        (a, b, c) = line.split()
        results = model_gigaword.most_similar(positive=[a.lower(), c.lower()], negative=[b.lower()])
        print(f"{a.lower()}-{b.lower()}+{c.lower()}={results[0][0]}")
        output.append(results[0][0])
print(output)

athens-greece+iraq=baghdad
athens-greece+thailand=bangkok
athens-greece+china=beijing
athens-greece+germany=berlin
athens-greece+switzerland=zurich
athens-greece+egypt=cairo
athens-greece+australia=sydney
athens-greece+vietnam=hanoi
athens-greece+cuba=havana
athens-greece+finland=helsinki
[': capital-common-countries', 'baghdad', 'bangkok', 'beijing', 'berlin', 'zurich', 'cairo', 'sydney', 'hanoi', 'havana', 'helsinki']


## Evaluate the default output

In [64]:
from analogy_check import precision
with open(os.path.join('../data','reference','dev.out'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
print("Score={:.2f}".format(100*precision(ref_data, output)))

Score=80.00


In [65]:

for i, word in enumerate(model_gigaword.index_to_key[:5]):
    print(f"Word: {word}")
    print(f"Vector: {model_gigaword[word]}")
    print()

Word: the
Vector: [-0.038194 -0.24487   0.72812  -0.39961   0.083172  0.043953 -0.39141
  0.3344   -0.57545   0.087459  0.28787  -0.06731   0.30906  -0.26384
 -0.13231  -0.20757   0.33395  -0.33848  -0.31743  -0.48336   0.1464
 -0.37304   0.34577   0.052041  0.44946  -0.46971   0.02628  -0.54155
 -0.15518  -0.14107  -0.039722  0.28277   0.14393   0.23464  -0.31021
  0.086173  0.20397   0.52624   0.17164  -0.082378 -0.71787  -0.41531
  0.20335  -0.12763   0.41367   0.55187   0.57908  -0.33477  -0.36559
 -0.54857  -0.062892  0.26584   0.30205   0.99775  -0.80481  -3.0243
  0.01254  -0.36942   2.2167    0.72201  -0.24978   0.92136   0.034514
  0.46745   1.1079   -0.19358  -0.074575  0.23353  -0.052062 -0.22044
  0.057162 -0.15806  -0.30798  -0.41625   0.37972   0.15006  -0.53212
 -0.2055   -1.2526    0.071624  0.70565   0.49744  -0.42063   0.26148
 -1.538    -0.30223  -0.073438 -0.28312   0.37104  -0.25217   0.016215
 -0.017099 -0.38984   0.87424  -0.72569  -0.51058  -0.52028  -0.1459
  0

## Documentation

Write some beautiful documentation of your program here.

## Analysis

Do some analysis of the results. What ideas did you try? What worked and what did not?

In [66]:

output = {}
inputs = {}
with open(os.path.join('../data', 'input', 'test.txt')) as fh:
    for i, line in enumerate(fh):
        line = line.strip()
        if line[0] == ':':
            get_pairs = {}
            output[line] = get_pairs
            ints = []
            inputs[line] = ints
            continue
        (a, b, c) = line.split()
        get_pairs[a]=b
        ints.append((a,b,c))
sels = [': capital-world', ': gram6-nationality-adjective',': gram7-past-tense',': gram8-plural',': gram9-plural-verbs']
oo = {': capital-world':[], ': gram6-nationality-adjective':[],': gram7-past-tense':[],': gram8-plural':[],': gram9-plural-verbs':[]}
capital_world_out = []
for sel in sels:
    for _,_,ech in inputs[sel]:
        ref_dict =output[sel]
        if ech in ref_dict:
            capital_world_out.append(ref_dict[ech])
        else:
            swapped_dict = {value: key for key, value in ref_dict.items()}
            oo[sel].append(swapped_dict[ech])
        
with open('../data/reference/test.txt', 'w') as file:
    for sel in sels:
        file.write(f"{sel}\n")
        test_gt = oo[sel]
        for item in test_gt:
            file.write(f"{item}\n")

In [67]:
# Analysing Failure cases

In [68]:
pds = []
with open('../output/test.out', 'r') as file:
    for i, line in enumerate(file):
        if line[0] == ":":
            continue
        pds.append(line)
gts = []
with open('../data/reference/test.txt', 'r') as file:
    for i, line in enumerate(file):
        if line[0] == ":":
            continue
        gts.append(line)
inp = []
with open('../data/input/test.txt', 'r') as file:
    for i, line in enumerate(file):
        if line[0] == ":":
            continue
        inp.append(line)

In [69]:
cor = 0
for i in range(len(inp)):
    if pds[i].strip() == gts[i].strip().lower():
        cor +=1
    else:
        print("(",inp[i].strip(),")", pds[i].strip(), gts[i].strip())

( Amman Jordan Kazakhstan ) kazakstan Astana
( Ankara Turkey Kazakhstan ) kazakstan Astana
( Antananarivo Madagascar Denmark ) krone Copenhagen
( Apia Samoa Kazakhstan ) kazakstan Astana
( Ashgabat Turkmenistan Denmark ) krone Copenhagen
( Astana Kazakhstan Germany ) german Berlin
( Astana Kazakhstan Switzerland ) swiss Bern
( Astana Kazakhstan Australia ) australian Canberra
( Astana Kazakhstan Moldova ) moldovan Chisinau
( Astana Kazakhstan Denmark ) krone Copenhagen
( Astana Kazakhstan Ireland ) irish Dublin
( Baghdad Iraq Australia ) australian Canberra
( Baku Azerbaijan China ) chinese Beijing
( Baku Azerbaijan Switzerland ) swiss Bern
( Baku Azerbaijan Australia ) australian Canberra
( Belgrade Serbia Switzerland ) swiss Bern
( Belmopan Belize Hungary ) forint Budapest
( Belmopan Belize Denmark ) krone Copenhagen
( Belmopan Belize Peru ) peruvian Lima
( Bishkek Kyrgyzstan Australia ) australian Canberra
( Bishkek Kyrgyzstan Denmark ) krone Copenhagen
( Caracas Venezuela Denmark )

In [70]:
cor/len(inp)

0.9260495700556398

In [71]:
a,b,c = "run","ran", "sing"
model_gigaword.most_similar(positive=[a.lower(), c.lower()], negative=[b.lower()])

[('perform', 0.6210198402404785),
 ('sang', 0.5608734488487244),
 ('tune', 0.5531638264656067),
 ('sung', 0.5515801310539246),
 ('sings', 0.5453202128410339),
 ('pray', 0.5444638133049011),
 ('singing', 0.5441002249717712),
 ('do', 0.5438511967658997),
 ('hear', 0.538349986076355),
 ('listen', 0.5350242853164673)]

In [72]:
import re
import copy
isNumber = re.compile(r'\d+.*')

def norm_word(word):
  if isNumber.search(word.lower()):
    return '---num---'
  elif re.sub(r'\W+', '', word) == '':
    return '---punc---'
  else:
    return word.lower()
      
def read_lexicon(filename):
  lexicon = {}
  for line in open(filename, 'r'):
    words = line.lower().strip().split()
    lexicon[norm_word(words[0])] = [norm_word(word) for word in words[1:]]
  return lexicon

In [73]:
lexicons = read_lexicon("../data/train/train/from_wikipedia.txt")

In [74]:
Q_hat = {word: model_gigaword[word] for word in model_gigaword.index_to_key}
Q = copy.deepcopy(Q_hat)

In [75]:
print("lexicons : ",len(lexicons.keys()))
print("Q_hat : ",len(Q_hat.keys()))
found = set(lexicons.keys()).intersection(set(Q_hat.keys()))
not_found = set(lexicons.keys()).difference(set(Q_hat.keys()))
print("found : ", len(lexicons.keys())-len(not_found))
print("not found : ", len(not_found))

lexicons :  474
Q_hat :  400000
found :  474
not found :  0


In [76]:
Q_hat_keys = set(Q_hat.keys())
epoch = 10
alpha = 1
beta = 1

for ep in range(epoch):
    print("epoch : ",ep)
    for idx,word in enumerate(found):
        print(idx,end='\r')
        word_synset = set(lexicons[word]).intersection(Q_hat_keys)
        if len(word_synset) == 0:
            continue
            
        qi = np.array([Q[ws] for ws in word_synset])
        sum_qi = np.sum(qi,axis = 0)
    
        Q[word] = ((beta*sum_qi) + (alpha*Q_hat[word]))/((beta*len(qi))+alpha)
    
    
    

epoch :  0
epoch :  1
epoch :  2
epoch :  3
epoch :  4
epoch :  5
epoch :  6
epoch :  7
epoch :  8
epoch :  9


In [77]:
def save_word_vectors(word_vecs, filename):
    num_words = len(word_vecs.keys())
    vector_size = len(word_vecs['the'])
    
    with open(filename, 'w') as f:
        f.write(f"{num_words} {vector_size}\n")
        
        for word in word_vecs.keys():
            vector = word_vecs[word]
            vector_str = ' '.join(map(str, vector))
            f.write(f"{word} {vector_str}\n")

In [78]:
save_word_vectors(Q,"./retrofit_new_vectors.txt")

In [79]:
output = []
not_working = []
working = []

retrofitted_model = KeyedVectors.load_word2vec_format('./retrofit_new_vectors.txt', binary=False)

with open(os.path.join('../data', 'input', 'dev.txt')) as fh:
    for i, line in enumerate(fh):
        line = line.strip()
        if line[0] == ':':
            output.append(line)
            continue
        (a, b, c) = line.split()
        # results = model_gigaword.most_similar(positive=[a.lower(), c.lower()], negative=[b.lower()])
        results = retrofitted_model.most_similar(positive=[a.lower(), c.lower()], negative=[b.lower()])
        output.append(results[0][0])

with open(os.path.join('../data','reference','dev.out'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
print("Score={:.2f}".format(100*precision(ref_data, output)))

Score=71.43


In [80]:
output = []
not_working = []
working = []

with open(os.path.join('../data', 'input', 'test.txt')) as fh:
    for i, line in enumerate(fh):
        line = line.strip()
        if line[0] == ':':
            output.append(line)
            continue
        (a, b, c) = line.split()
        # results = model_gigaword.most_similar(positive=[a.lower(), c.lower()], negative=[b.lower()])
        results = retrofitted_model.most_similar(positive=[a.lower(), c.lower()], negative=[b.lower()])
        output.append(results[0][0])

with open(os.path.join('../data','reference','test.txt'), 'rt') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
print("Score={:.2f}".format(100*precision(ref_data, output)))

Score=92.60


In [81]:
# wordnet-synonyms+.txt
Train = 13.92
Test = 41

# wordnet-synonyms+.txt (10)
Train = 14.93
Test = 43.63

# wordnet-synonyms.txt (10)
Train = 18.72
Test = 53.05

# ppdb-xl.txt (10)
Train = 24.33
Test = 47.23

# framenet.txt (10)
Train = 16.44
Test = 54.70

# from_wikipedia.txt (10)
Train = 71.42
Test =  92.60